In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.16.0 to work with agogemls


In [2]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails
from azureml.core import Experiment

# Get the training dataset
titanic_ds = ws.datasets.get("Titanic")

# Set the script parameters
script_params = {
    '--regularization': 0.45
}

training_folder = 'titanic-training'
cluster_name = "d12compute"

# Create an estimator
estimator = SKLearn(source_directory=training_folder,
                    entry_script='titanic_training.py',
                    script_params=script_params,
                    compute_target=cluster_name,
                    inputs=[titanic_ds.as_named_input('Titanic')], # Pass the Dataset object as an input...
                    pip_packages=['azureml-dataprep[pandas]'] # ...so you need the dataprep package
                    )

# Create an experiment
experiment_name = 'Titanic-experiment-3'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)

# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'Titanic-experiment-3_1603284189_8d5a9bf3',
 'target': 'd12compute',
 'status': 'Completed',
 'startTimeUtc': '2020-10-21T12:43:54.217437Z',
 'endTimeUtc': '2020-10-21T12:44:51.913942Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '2b6ec22e-1a36-4b8c-86e5-3ed3af51d108',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '6256dd80-b31c-4d77-8316-d962ab3f41c1'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'Titanic', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'titanic_training.py',
  'command': [],
  'useAbsolutePath': False,
  'arguments': ['--regularization', '0.45'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'd12compute',
  'dataReferences': {},
  'data': {'Titanic': {'dataLocation': {'dataset': {'id': '6256dd80-b31c-4d77-8316-d962ab3f

In [3]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.45
Accuracy 0.7985074626865671
AUC 0.8230025183150182


azureml-logs/55_azureml-execution-tvmps_d921636f4cf583c945181140d9d19b44a0bece649dd57bb951cf9ed182a7f2f6_d.txt
azureml-logs/65_job_prep-tvmps_d921636f4cf583c945181140d9d19b44a0bece649dd57bb951cf9ed182a7f2f6_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_d921636f4cf583c945181140d9d19b44a0bece649dd57bb951cf9ed182a7f2f6_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/126_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
logs/azureml/dataprep/engine_spans_l_83657aca-f113-409b-81b6-aec735eb1458.jsonl
logs/azureml/dataprep/python_span_l_83657aca-f113-409b-81b6-aec735eb1458.jsonl
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
outputs/titanic_model.pkl


In [4]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/titanic_model.pkl', model_name='Titanic',
                   tags={'Training context':'Remote Estimator'},
                   properties={'Regularization Rate': run.get_metrics()['Regularization Rate'],
                       'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Model(workspace=Workspace.create(name='agogemls', subscription_id='da21a094-26a3-472f-991b-e2b11979af40', resource_group='agoge'), name=Titanic, id=Titanic:5, version=5, tags={'Training context': 'Remote Estimator'}, properties={'Regularization Rate': '0.45', 'AUC': '0.8230025183150182', 'Accuracy': '0.7985074626865671'})